In [1]:
%matplotlib notebook

### BARPA-R Future Tropical Low Case Study

In a warmer climate, moist thermodynamics imply that the atmosphere will be able to carry more moisture.

This means that rainfall events have the potential to become more intense in the future.

While CMIP6 models are able to simulate the increasing levels of atmospheric moisture, downscaled projections are needed to see the effect on small-scale weather systems that global models do not resolve.

This case-study looks at BARPA-R downscaling of EC-Earth3. Of all models downscaled with BARPA-R, EC-Earth3 shows the largest increase in atmospheric moisture.

We have identified two tropical low events from BARPA-R-EC-Earth3 with similar tracks paths and minimum pressures. 

Follow the tutorial to see how the rainfall and atmospheric moisture vary between the two lows.


In [2]:
# First lets import some libraries
import os, sys
os.chdir("/home/548/eh6215/python/BARRA2_evaluation/jt/notebooks/")
import loaddata
import loaddata_staged

from glob import glob
import xarray as xr
import pandas
import cartopy.crs as ccrs
from dask.distributed import Client, LocalCluster
from matplotlib import pyplot as plt
import datetime as dt


In [3]:
# Let's explictly load dask so we can check progress
# Copy and past the dashboard link/path from this cell's output
# to the Dask tab on the left.
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43997,Workers: 2
Dashboard: /proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:41255,Total threads: 1
Dashboard: /proxy/44917/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:34375,


In [5]:
# In order to highlight the rich data available from BARPA, this tutorial features animations
# The following settings, as well as "%matplotlib notebook" at the beginning of the note-book, allow these animations to run


from matplotlib.widgets import Slider
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
#plt.rcParams["animation.embed_limit"] = 60 # MB
plt.ioff()

In [7]:
# Now, let's load our first tropical low
# Remember this is a synthetic, model-generated track, it didn't occur in the real world!
# the variables we're loading are:
#   psl: sea level pressure
#   pr: precipitation
#   prw: atmospheric total column water vapour
hist = {}
for var in ['psl','pr','prw']:
    hist[var] = loaddata.load_barpa_data("BARPA-R",
                         "EC-Earth3",
                         "historical",
                         "1hr",
                         var,
                         tstart = dt.datetime(2009,1,5),
                         tend=dt.datetime(2009,1,16),
                         lonrange = (105,160),
                         latrange = (-45,-7))[var]

In [8]:
# let's check these are what I say they are, and their units:
for var in hist: 
    print(f"{var:3s}: {hist[var].long_name:25s} ({hist[var].attrs['units']})")


psl: Sea Level Pressure        (Pa)
pr : Precipitation             (kg m-2 s-1)
prw: Water Vapor Path          (kg m-2)


In [9]:
# Now we load the second low

future = {}
for var in ['psl','wsgsmax','pr','huss','prw']:
    future[var] = loaddata_staged.load_barpa_data("BARPA-R",
                         "EC-Earth3",
                         "ssp370",
                         "1hr",
                         var,
                         tstart = dt.datetime(2088,12,25),
                         tend=dt.datetime(2089,1,6),
                         lonrange = (105,160),
                         latrange = (-45,-7))[var]

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/iris/fileformats/_nc_load_rules/helpers.py:431: UnknownCellMethodWarning: NetCDF variable 'wsgsmax' contains unknown cell method 'interpolation'
  warnings.warn(message=msg, category=UnknownCellMethodWarning)


In [10]:
# Now, let's change those precipitation units into something more understandable
# Only execute this cell once!

hist['pr'][:] = hist['pr'][:]*3600
hist['pr'].attrs['units']='mm hr-1'
future['pr'][:] = future['pr'][:]*3600
future['pr'].attrs['units']='mm hr-1'


In [11]:
# to start with, we'll look at 12-hourly data. You can increase the frequency later (by rerunning the whole notebook) if you'd lke

for var in hist:
    hist[var] = hist[var].resample(time='12h').mean()
    future[var] = future[var].resample(time='12h').mean()

In [12]:
# Now let's find those lows!
# In this example, the systems we're interested in are the lowest-pressure 
# objects in the box 100 E-150 E, 30S - 0S. 
# So we can just find the minimum over space at each timestep to create the track paths
track_x, track_y = [0,0],[0,0]
for i,ds in enumerate([hist,future]):
    # first define initial figure
    track_x[i] = ds['psl'].sel(lon=slice(100,150)).lon[ds['psl'].sel(lat=slice(-30,0),lon=slice(100,150)).min(['lat']).argmin('lon')]
    track_y[i] = ds['psl'].sel(lat=slice(-30,0)).lat[ds['psl'].sel(lat=slice(-30,0),lon=slice(100,150)).min(['lon']).argmin('lat')]

To start off, we'll create an animation of total column water, with precipitation and the track path drawn on top.
Precip will be masked at rates of <2mm/hour so the prw is visible underneath. 


In [ ]:

# to create an animation we first create the initial frame. 

# set up the figure
fig=plt.figure(figsize=(12,4)) # set up figure

# empty objects for the animated components
ax,pcolor1,pcolor2,dot = [0,0],[0,0],[0,0],[0,0]

# populate the initial frame
for i,ds in enumerate([hist,future]):
    ax[i]=plt.subplot(1,2,i+1,projection=ccrs.PlateCarree()) #set up axes with geometric projection
    pcolor1[i] = (ds['prw'][0]).plot(vmin=0,vmax=100,cmap='Purples',add_colorbar=False) # plot initial hourly precip rate, with sec-> hour conversion rate
    pcolor2[i] = (ds['pr'][0].where(ds['pr'][0]>2)).plot(vmin=2,vmax=50,cmap='Blues',add_colorbar=False) # plot initial hourly precip rate, with sec-> hour conversion rate
    ax[i].coastlines() # draw coastlines
    ax[i].set_xlim(105,160)
    ax[i].set_ylim(-45,-7)
    ax[i].set_title(ds['prw'].time[0].values.__str__()[:16]) # set timestamp as axes label
    ax[i].plot(track_x[i][:-1],track_y[i][:-1],'k') # plot full cyclone track
    dot[i] = plt.plot(track_x[0],track_y[0],'ko') # plot initial cyclone location

fig.tight_layout()

# second, we create a function that will update the figure to time-step t
def animate(t): 
    for i,ds in enumerate([hist,future]):
        pcolor1[i].set_array(ds['prw'][t].values) # update rainfall to time t
        pcolor2[i].set_array(ds['pr'][t].where(ds['pr'][t]>10).values) # update rainfall to time t
        dot[i][0].set_xdata([track_x[i][t]]) # update cyclone lon to time t
        dot[i][0].set_ydata([track_y[i][t]]) # update cyclone lat to time t
        ax[i].set_title(ds['prw'].time[t].values.__str__()[:16]) # update title

# finally, we use FuncAnimation to generate our figure


timesteps = min(len(hist['pr'].time), len(future['pr'].time))
matplotlib.animation.FuncAnimation(fig, animate, frames=timesteps)  # generate animation


# this may take a while!

Congrats, you've completed the first runthrough of the notebook. Now, get creative. You might like to:

Simple extensions:
* Increase the update frequency from 12 hours to 3 hours
* Add colorbars
* Look at some other variables: consider
    * maximum wind gust (wsgwmax)
    * Temperature (tas)
    * Clouds (cll/clm/clt, or rlut is a good proxy)
    * 6-hourly Winds on pressure levels (uaXXX, vaXXX, where XXX is the pressure level)
* Generate some still figures that summarise your key findings from this example
 
More complicated extensions:
* Try to add wind barbs, contours or streamfunctions
* Adapt to look at the 3D structure of the system
* Compute dewpoint temperatures, or moist static energy

In [17]:
# A hint:
# Jupyter currently doesn't allow both animations and still figures to be displayed in the same notebook
# the following work around is needed to display stills:


fig = plt.figure(figsize=(3,3))
plt.plot([1,2,3],[4,5,6],'.')

def animate(t):
    return
    
matplotlib.animation.FuncAnimation(fig, animate, frames=1)  # generate animation

# alternatively, copy to a new notebook and remove the line
# %matplotlib notebook